In [1]:
# Twitter Streaming API
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream

# API Key & Token Info
import const

# Kafka
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.errors import KafkaError

import json

In [2]:
class Producer():
    def __init__(self, bootstrap_servers):
        try:
            self.producer = KafkaProducer(bootstrap_servers=bootstrap_servers,
                                          max_block_ms = 10000,
                                          retries = 0, 
                                          acks = 1, 
                                          value_serializer = lambda v: json.dumps(v).encode("utf-8"))
        except KafkaError as exc:
            print(f"kafka producer - Exception during connecting to broker - {exc}")
            return False
    
    def stop(self):
        self.producer.close()

    def send_data(self, topic, data):        
        # Asynchronous by default
        self.producer.send(topic, data).add_callback(self.on_send_success).add_errback(self.on_send_error)
        
        # block until all async messages are sent
        self.producer.flush()
    
    def on_send_success(self, record_metadata):
        print("**********Send Success***********")
        print("record_metadata.topic: ", record_metadata.topic)
        print("record_metadata.partition: ", record_metadata.partition)
        print("record_metadata.offset: ", record_metadata.offset)
        pass

    def on_send_error(self, excp):
        print("**********Send Error Occur**********")
        log.error("I am an errback", exc_info=excp)

In [3]:
# Read the credentials form const
CONSUMER_KEY = const.CONSUMER_KEY
CONSUMER_SECRET = const.CONSUMER_SECRET
ACCESS_TOKEN = const.ACCESS_TOKEN
ACCESS_SECRET = const.ACCESS_SECRET

In [4]:
class TweetsListener(Stream):
    def __init__(self, producer, topic_name, **kwargs):
        super(TweetsListener, self).__init__(**kwargs)
        self.producer = producer
        self.topic_name = topic_name
        
    def on_data(self, data):
        try:
            msg = json.loads(data)
            
            if not msg['truncated']:
                text = msg['text']
            else:
                text = msg['extended_tweet']['full_text']

            print(text)
            
            self.producer.send_data(self.topic_name, text)

        except BaseException as e:
            print(f"error: {str(e)}")

        return True
    
    def on_error(self, status):
        print(status)
        
        return True

In [5]:
# Create Kafka Producer 
bootstrap_servers = ['localhost:9092'] # kafka broker ip
topic_name = 'tweets' # kafka topic name
kafka_producer = Producer(bootstrap_servers=bootstrap_servers)

In [6]:
# Send tweets to Kafka Broker
twitter_stream = TweetsListener(consumer_key=CONSUMER_KEY,
                                consumer_secret=CONSUMER_SECRET,
                                access_token=ACCESS_TOKEN,
                                access_token_secret=ACCESS_SECRET, 
                                producer=kafka_producer, 
                                topic_name=topic_name)

twitter_stream.sample()

error: 'truncated'
error: 'truncated'
error: 'truncated'
みんな起きて
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28240
E diga-se de passagem um rolão.
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28241
RT @mingkigen: 🥺 https://t.co/QRGcbqhMwr
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28242
RT @BTSPublicity: If you haven't cast your daily 25 votes, drop your votes here for @BTS_twt 🗳️

I vote for #BTS as #TheGroup at the 2021 #…
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28243
@nosugar_redbean 바에단같죠? 실리안은 저렇게 이마가 훤하지 않을거에요..... ㅋㅋㅋㅋㅋㅋㅋㅋ
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28244
RT @EJkM488pMu5MdLS: 三点リーダー（……）

error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
マジでかわいくてめちゃくちゃ噛み締めてる、、、、、、、、大感謝
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28299
RT @caIebgod: @sufrimiento69 es tab tierno voy a mworir https://t.co/EjWfxsQLxQ
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28300
RT @swppb12: ทุกคนคะตอนนี้ดูไลฟ์ไหนกันคะ #ม็อบ29ตุลา64
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28301
RT @_BBOPEEP: แอ๋ว : https://t.co/ZDs7EELvrk
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28302
RT @MinistryOfEXO: #NNG2ndAnniversary ♥️🎉 https://t.co/yJdbt64w4o
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.part

ہم ڈسٹرک ساوتھ کے صدر #خلیل_ہوت کو کامیاب احتجاجی مظاہرہ آرگنائز کرنے پر #دلی_مبارک باد پیش کرتے ہیں امید کرتے ہیں کہ #خلیل_ہوت #چیئرمین_بلاول_بھٹو کے دست بازو بنکر ہمیشہ عوام دشمن سلیکٹڈ حکومت کے سامنے کردار ادا کرینگے
@khalil_hoat
#MehngaPakistanNaManzoor https://t.co/FWzN12Kr4W
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28346
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
MODの略称でいわゆる拡張データであり、Minecraftにさまざまな要素を追加する。
Minecraftは比較的MODが作りやすいゲームであるため、フォーラムでは日々大型の変更を加えるものから小規模の変更を加えるものまで、新しいMODが数多く作られている
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28347
…やめとこ
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28348
アーニャって名前がもう可愛くない？？？(既に末期症状)
**********Send Success***********
record_metadata.topic:  tweet

error: 'truncated'
error: 'truncated'
早寝しなきゃいけなかったのにコーヒー効いちゃってて眠気がお留守～💦
どうしよう😖💦
何日か粉末切らしてたから嬉しくてつい…(あほの子全開)
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28409
RT @bckupacc99: November will be kind. November will be magical. November will bring the missing energy. November will be working towards o…
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28410
RT @treasuremembers: 쟁형잉강찡엉중상징🦁
#JIHOON #지훈 #ジフン https://t.co/jkddVJMlkQ
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28411
おわぁー、ヴァイオレットエヴァーガーデン全部見たー
まさか通しで最後まで見るとは自分でも思わんかった
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28412
RT @warnettebot: “I love you,” she whispers. “I love you so much.”
“That you could love me at all seems like some kind of m

error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
神激のぬい欲しすぎて前出ないかなってずっと考えててないなら自分で作っちゃえ！！精神で抜い抜いしてたくらいには望んでた、、、
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28453
อินดี้หล่อมากอ้ากแคแีแด่่ก่ก่กกตำจำ
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28454
あのマジでなにw
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28455
arwah goyang karawang is just black swan indonesian ver.
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28456
Necesito verla
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  28457
RT @nuttahx_: 🌹: ถ่ายให้ติดคนข้างหลังนะ
🐱: คนไหน https://t.co/Wsy8vHzSHE
**********Send Success*

...

error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
「天下無敵」その言葉　
心の奥に刻み込み　　
我等立つ前敵は無く　
今夜も爆音響かせる　
一生一度の人生を　
飾ってみせますこの街で　
十代最後の親不孝　
許した親に感謝して
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29125
玉！玉！竿！ボッキ！スタァンドゥアアアアップ！
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29126
果てない空がそこにあるって いま確かな声が聞こえる 飛べない自分を変えていこうか 踏み出して何度でも やり直そう 力強く 「果てない空」
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29127
知識は多いに越した事は無い。だが本当に大切なのは、それを適切に扱う事ができる知性の存在だ。 ―"枢動者" レヴァリエ・アレクサンドリア
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29128
error: 'truncated'
RT @Grandelovesick2: .@BLACKPINK #LISA

Vessels, Barges, and Marine Related Equipment!⚓ - https://t.co/NxXF05zyNM https://t.co/e2Nbyt4Y4b
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29162
error: 'truncated'
error: 'truncated'
error: 'truncated'
Did you know, that frogs actually swallow with their eyes?
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29163
１９９８～２００３に放送されたゲーム番組gamewave。その全編を視聴終了。年４８回放送×５年で240話。１２０時間の超大作だった。ファミ通との連携が濃い番組で、濃いゲーム情報と浜村通信×伊集院光の「浜さん光」は浜村通信のあまりの事情通と伊集院の古ゲーム知識が最高だった。
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29164
国1ほとんど白線消えかけてるから
夜めっちゃ見づらくなってるから
線引き直してほしいわ(笑)
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29165
RT @shogonshogooo1: 今やらないと損。
月100万稼ぐために絶対やった方が良い。あと2年以内で大手の会社も副業禁止が解禁。そうなればプロフの副業が人気になるのはわかっている。そ

error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
error: 'truncated'
RT @SamuelTabitha3: Comment using “❤️” and follow everyone that likes your comment!! It's proh moment😅🔙.. 🤩💜
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29208
RT @cheewitgu: หลายบริษัทจะให้กลับไปทำงานที่ออฟฟิศเร็วๆนี้ หลายคนเริ่มเครียดโดยไม่รู้ตัว เพราะบางงานมันทำที่บ้านได้ งานไหนต้องเข้าออฟฟิศค่อ…
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29209
Mais um dia agradecendo a Deus e o Universo pelo trabalho que tenho.

Cara eu sou mto feliz com esse trabalho mds
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  29210
RT @afroparadiso: https://t.co/89ZdQ6oKRD
**********Send Success***********
record_metadata.topic:  tweets
record_metadata.partition:  0
record_metadata.offset:  2

error: 'truncated'
error: 'truncated'


KeyboardInterrupt: 